In [104]:
from elasticsearch import Elasticsearch

client = Elasticsearch(
    "https://f0c40d3bde77471f83e6cace87537665.ap-southeast-1.aws.found.io",
    # api_key="Mk9odVlwY0JiN3JESEE4VzhkZ286WTNuZ29WR2pYMDF6em1DV193WVdkZw=="
    api_key="MnVpVVlwY0JiN3JESEE4V2FOaEk6X2NRRS1ESnM2V1A2TDJKQ25pb01NUQ=="
)

index_name = "cvedocs"

# Delete if exists (optional cleanup step for testing)
if client.indices.exists(index=index_name):
    client.indices.delete(index=index_name)

# Create index with proper mapping
mappings = {
    "mappings": {
        "properties": {
            "text": {
                "properties": {
                    "cve_id": {"type": "keyword"},
                    "description": {"type": "text"},
                    "severity": {"type": "keyword"}
                }
            }
        }
    }
}

# mappings = {
#     "mappings": {
#         "properties": {
#             "text": {"type": "text"},  # not an object
#             "metadata": {
#                 "properties": {
#                     "cve_id": {"type": "keyword"},
#                     "severity": {"type": "keyword"},
#                     "cvss_score": {"type": "float"},
#                     "fix_version": {"type": "keyword"},
#                     "published_date": {"type": "date"},
#                     "last_updated": {"type": "date"},
#                     # Add more fields as needed
#                 }
#             }
#         }
#     }
# }

mappings = {
    "mappings": {
        "properties": {
  "text": { "type": "text" },
  "vector": {
    "type": "dense_vector",
    "dims": 768,
    "index": True,
    "similarity": "cosine"
  },
  "metadata": {
    "properties": {
      "cve_id": { "type": "keyword" },
      "severity": { "type": "keyword" },
      "cvss_score": { "type": "float" },
      "fix_version": { "type": "keyword" },
      "published_date": { "type": "date" },
      "last_updated": { "type": "date" }
    }
  }
}}}
client.indices.create(index=index_name, body=mappings)

# Sample document
# doc = {
#     "text": {
#         "cve_id": "CVE-2024-12345",
#         "description": "This is a test CVE vulnerability.",
#         "severity": "High"
#     }
# }

# Index the document
# response = client.index(index=index_name, document=doc)
# print("Document indexed:", response)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'cvedocs'})

POST /_security/api_key
{
  "name": "my-api-key1",
  "role_descriptors": {
    "my-role": {
      "cluster": ["all"],
      "index": [
        {
          "names": ["cyberdocs"],
          "privileges": ["all"]
        }
      ]
    }
  }
}

In [28]:
from langchain.embeddings import OllamaEmbeddings
from elasticsearch import Elasticsearch
import ollama
# MODEL_NAME = "all-MiniLM-L6-v2"  # small and fast
MODEL_NAME='nomic-embed-text'
ES_INDEX = "cvedocs"
# model = SentenceTransformer(MODEL_NAME)
# model = OllamaEmbeddings(model=MODEL_NAME)

def ollama_embedding_function(texts):
    """
    texts: List[str] → List[List[float]]
    """
    if isinstance(texts, str):
        texts = [texts]
    embeddings = []
    for text in texts:
        response = ollama.embeddings(model="nomic-embed-text", prompt=text)
        embeddings.append(response['embedding'])
    return embeddings[0]




In [ ]:
from langchain.embeddings import OllamaEmbeddings
from elasticsearch import Elasticsearch
import ollama
# MODEL_NAME = "all-MiniLM-L6-v2"  # small and fast
MODEL_NAME='nomic-embed-text'
ES_INDEX = "cvedocs"
# model = SentenceTransformer(MODEL_NAME)
# model = OllamaEmbeddings(model=MODEL_NAME)

def ollama_embedding_function(texts):
    """
    texts: List[str] → List[List[float]]
    """
    if isinstance(texts, str):
        texts = [texts]
    embeddings = []
    for text in texts:
        response = ollama.embeddings(model="nomic-embed-text", prompt=text)
        embeddings.append(response['embedding'])
    return embeddings[0]




In [94]:
from sentence_transformers import SentenceTransformer
sentences = ["Attacker takes a screenshot", "Attacker captures the screen"]
model = SentenceTransformer('basel/ATTACK-BERT')

def embedding_attack_bert(text):
    return model.encode(text)    
# embeddings = model.encode(sentences[0])

C:\Users\P0107146\AppData\Local\anaconda3\envs\py312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P0107146\AppData\Local\anaconda3\envs\py312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P0107146\AppData\Local\anaconda3\envs\py312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-war

In [105]:
import re
import html
from urllib.parse import unquote


def parse_cpe(cpe_uri):
    """
    Parse a CPE 2.3 URI and return a dictionary of components.
    Example: cpe:2.3:a:vendor:product:version:*:*:*:*:*:*:*
    """
    parts = cpe_uri.split(":")
    if len(parts) >= 7:
        return {
            "type": parts[2],         # a / o / h
            "vendor": unquote(parts[3]),
            "product": unquote(parts[4]),
            "version": unquote(parts[5])
        }
    return {}


def extract_cve_data(json_data, embed_model=None):
    try:
        cve_id = json_data.get("cveMetadata", {}).get("cveId", "")
        containers = json_data.get("containers", {})
        cna = containers.get("cna", {})

        # Description
        description = next(
            (d.get("value") for d in cna.get("descriptions", []) if d.get("lang") == "en"),
            ""
        )
        description = html.unescape(description.strip())

        # CVSS metrics
        metrics = cna.get("metrics", [])
        cvss = {}
        for m in metrics:
            if "cvssV3_1" in m:
                cvss = m["cvssV3_1"]
                break

        # CWE
        cwes = []
        for pt in cna.get("problemTypes", []):
            for desc in pt.get("descriptions", []):
                if desc.get("lang") == "en":
                    cwes.append({
                        "id": desc.get("cweId"),
                        "description": desc.get("description")
                    })

        # Affected packages
        affected_packages = []
        for a in cna.get("affected", []):
            vendor = a.get("vendor", "")
            product = a.get("product", "")
            version_info = a.get("versions", [])
            for v in version_info:
                pkg = {
                    "vendor": vendor,
                    "product": product,
                    "version": v.get("version"),
                    "lessThanOrEqual": v.get("lessThanOrEqual"),
                    "version_type": v.get("versionType"),
                    "status": v.get("status")
                }
                # Optionally include parsed CPE info if available
                for cpe_entry in a.get("cpe", []):
                    cpe_data = parse_cpe(cpe_entry.get("cpe23Uri", ""))
                    pkg.update(cpe_data)
                affected_packages.append(pkg)

        # Infer fix versions
        fix_versions = sorted(
            {f"> {pkg['lessThanOrEqual']}" for pkg in affected_packages if pkg.get("lessThanOrEqual")}
        )
        fix_version = fix_versions[0] if fix_versions else None

        # References
        references = [r.get("url") for r in cna.get("references", []) if "url" in r]

        # Timeline
        timeline = [
            {"date": t["time"], "event": t["value"]}
            for t in cna.get("timeline", []) if "time" in t and "value" in t
        ]

        # Credits
        credits = [c.get("value") for c in cna.get("credits", []) if c.get("lang") == "en"]

        # SSVC (CISA enrichment)
        ssvc_data = {}
        for adp in containers.get("adp", []):
            for metric in adp.get("metrics", []):
                if "other" in metric and metric["other"].get("type") == "ssvc":
                    content = metric["other"].get("content", {})
                    ssvc_data = {
                        "role": content.get("role"),
                        "version": content.get("version"),
                        "exploitStatus": content.get("exploitStatus")
                    }
                    for opt in content.get("options", []):
                        ssvc_data.update(opt)
                    break



        # Final output
        return {
            # "id": cve_id,
            "text": description,
            "vector": embed_model(description),
            "fulltext":json_data,
            "metadata": {
                "title": cna.get("title", ""),
                "published_date": json_data.get("cveMetadata", {}).get("datePublished", ""),
                "last_updated": json_data.get("cveMetadata", {}).get("dateUpdated", ""),
                "cvss_score": cvss.get("baseScore"),
                "cvss_vector": cvss.get("vectorString"),
                "severity": cvss.get("baseSeverity"),
                "cwe": cwes,
                "affected_packages": affected_packages,
                "fix_version": fix_version,
                "references": references,
                "timeline": timeline,
                "credits": credits,
                "assigner": json_data.get("cveMetadata", {}).get("assignerShortName"),
                "ssvc": ssvc_data
            },
            "sources": references
        }

    except Exception as e:
        print(f"Failed to parse CVE JSON: {e}")
        return {}

In [109]:
import os
from pathlib import Path
import json

def convert_all(input_dir: str, client, batch_size=500):
    input_path = Path(input_dir)
    all_json_files = list(input_path.rglob("*.json"))
    all_json_files=all_json_files
    print(f"[i] Found {len(all_json_files)} JSON files in '{input_dir}'")

    l1 = []
    total_processed = 0
    total_failed = 0

    for idx, file_path in enumerate(all_json_files):
        try:
            with file_path.open() as f:
                cve_json = json.load(f)
            doc = extract_cve_data(cve_json, embedding_attack_bert)

            # Skip REJECTED CVEs
            if doc.get("state", "").upper() == "REJECTED":
                continue

            l1.append({"index": {"_index": "cvedocs","_id":cve_json.get("cveMetadata", {}).get("cveId", "")}})
            l1.append(doc)
            total_processed += 1

            if len(l1) >= batch_size * 2:
                res = client.bulk(index="cvedocs", operations=l1, refresh=True)
                print(f"[✓] Bulk inserted {batch_size} documents")
                l1 = []  # Clear the batch

        except Exception as e:
            print(f"[!] Failed to process {file_path}: {e}")
            total_failed += 1

    # Final flush if anything remains
    if l1:
        res = client.bulk(index="cvedocs", operations=l1, refresh=True)
        print(f"[✓] Final bulk inserted {len(l1)//2} documents")

    print(f"[✔] Total processed: {total_processed}")
    print(f"[✗] Total failed: {total_failed}")

# Run the batch processor
convert_all(r"C:\Users\P0107146\Downloads\2025", client)


[i] Found 2829 JSON files in 'C:\Users\P0107146\Downloads\2025'
[!] Failed to process C:\Users\P0107146\Downloads\2025\0xxx\CVE-2025-0020.json: 'charmap' codec can't decode byte 0x9d in position 1092: character maps to <undefined>
[!] Failed to process C:\Users\P0107146\Downloads\2025\0xxx\CVE-2025-0109.json: 'charmap' codec can't decode byte 0x9d in position 23149: character maps to <undefined>
[!] Failed to process C:\Users\P0107146\Downloads\2025\0xxx\CVE-2025-0110.json: 'charmap' codec can't decode byte 0x9d in position 14929: character maps to <undefined>
[!] Failed to process C:\Users\P0107146\Downloads\2025\0xxx\CVE-2025-0111.json: 'charmap' codec can't decode byte 0x9d in position 22928: character maps to <undefined>
[!] Failed to process C:\Users\P0107146\Downloads\2025\0xxx\CVE-2025-0124.json: 'charmap' codec can't decode byte 0x9d in position 12104: character maps to <undefined>
[!] Failed to process C:\Users\P0107146\Downloads\2025\0xxx\CVE-2025-0125.json: 'charmap' codec c

KeyboardInterrupt: 

In [115]:
query = "List all the cve related Filesystem "
query_vector = embedding_attack_bert(query)

response = client.search(index="cvedocs", body={
    "size": 100,
    "knn": {
        "field": "vector",
        "query_vector": query_vector,
        "k": 100,
        "num_candidates": 1000
    }
})

# Display results with score and title
for hit in response["hits"]["hits"]:
    title = hit["_source"]["metadata"].get("title", "No title")
    print(f"{hit['_score']:.2f} - {title}")


0.83 - Expedition: Wildcard Expansion Vulnerability
0.78 - Microword eScan Antivirus Folder Watch List rtscanner removeExtraSlashes stack-based overflow
0.76 - JoeyBling bootplus SysFileController.java path traversal
0.76 - Grub2: ufs: integer overflow may lead to heap based out-of-bounds write when handling symlinks
0.76 - Information exposure vulnerability in PMB platform
0.76 - JoeyBling bootplus list sql injection
0.76 - Grub2: reiserfs: integer overflow when handling symlinks may lead to heap based out-of-bounds write when reading data
0.75 - MicroWord eScan Antivirus mwav.conf ReadConfiguration buffer overflow
0.75 - Grub2: romfs: integer overflow when handling symlinks may lead to heap based out-of-bounds write when reading dat
0.75 - JoeyBling bootplus list sql injection
0.75 - Grub2: commands/dump: the dump command is not in lockdown when secure boot is enabled
0.75 - Grub2: fs/hfs: integer overflow may lead to heap based out-of-bounds write
0.75 - Buffer overflow in some file

In [6]:
query = "CVE related to database"
query_vector = model(query)[0]  # Assuming model(query) returns a list or array of vectors

# Build the semantic search query
response = client.search(index="cvedocs", body={
    "size": 100,  # Adjust this based on how many results you want
    "query": {
        "script_score": {
            "query": {"match_all": {}},  # Apply cosine similarity on all documents
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",  # +1.0 to avoid negative scores
                "params": {"query_vector": query_vector}
            }
        }
    }
})

# Display the top results with title
for hit in response["hits"]["hits"]:
    title = hit["_source"]["metadata"].get("title", "No title")
    print(f"{hit['_score']:.2f} - {title}")





1.46 - CVE-2025-0035 
1.46 - CVE-2025-0049 Disclosure of sensitive information in an error message in GoAnywhere prior to version 7.8.0
1.45 - CVE-2025-0001 authenticated arbitrary file read vulnerability
1.45 - CVE-2025-0036 
1.45 - CVE-2025-0050 Mali GPU Userspace Driver allows an Out-of-Bounds access
1.44 - CVE-2025-0014 
1.41 - CVE-2025-0015 Mali GPU Kernel Driver allows improper GPU processing operations
1.39 - CVE-2025-0037 


In [7]:
response = client.search(index="cvedocs", body={
    "size": 10,
    "query": {
        "bool": {
            "filter": [
                {"term": {"metadata.severity": "HIGH"}},
            ]
        }
    }
})

for hit in response["hits"]["hits"]:
    print(f"{hit['_id']} - {hit['_source']['metadata']['severity']} - {hit['_source']['text']}")

CVE-2025-0014 - HIGH - CVE-2025-0014 Incorrect default permissions on the AMD Ryzen(TM) AI installation folder could allow an attacker to achieve privilege escalation, potentially resulting in arbitrary code execution.
CVE-2025-0035 - HIGH - CVE-2025-0035 Unquoted search path within AMD Cloud Manageability Service can allow a local attacker to escalate privileges, potentially resulting in arbitrary code execution.


In [8]:
query = " what are the exploits will impact disk "
query_vector = model(query)[0]

response = client.search(index="cvedocs", body={
    "size": 100,
    "query": {
        "script_score": {
            "query": {
                "bool": {
                    "filter": [
                        # {"term": {"metadata.severity": "CRITICAL"}},
                        # {"range": {"metadata.cvss_score": {"gte": 9.0}}}
                    ]
                }
            },
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'vector') + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }
})

print('Total Document Found : {}'.format(len(response['hits']['hits'])))
for hit in response["hits"]["hits"]:
    print(f"{hit['_id']} - {hit['_source']['metadata']['title']}")

Total Document Found : 8
CVE-2025-0014 - CVE-2025-0014 
CVE-2025-0015 - CVE-2025-0015 Mali GPU Kernel Driver allows improper GPU processing operations
CVE-2025-0036 - CVE-2025-0036 
CVE-2025-0035 - CVE-2025-0035 
CVE-2025-0001 - CVE-2025-0001 authenticated arbitrary file read vulnerability
CVE-2025-0050 - CVE-2025-0050 Mali GPU Userspace Driver allows an Out-of-Bounds access
CVE-2025-0037 - CVE-2025-0037 
CVE-2025-0049 - CVE-2025-0049 Disclosure of sensitive information in an error message in GoAnywhere prior to version 7.8.0


In [104]:
response['hits']

{'total': {'value': 29, 'relation': 'eq'},
 'max_score': 1.2515438,
 'hits': [{'_index': 'cve-index',
   '_id': 'CVE-2025-0066',
   '_score': 1.2515438,
   '_source': {'id': 'CVE-2025-0066',
    'text': 'Under certain conditions SAP NetWeaver AS for ABAP and ABAP Platform (Internet Communication Framework) allows an attacker to access restricted information due to weak access controls. This can have a significant impact on the confidentiality, integrity, and availability of an application',
    'vector': [-0.05567854270339012,
     0.048889148980379105,
     -0.05899299308657646,
     -0.007609989959746599,
     0.004149011801928282,
     -0.03386954218149185,
     0.06040226295590401,
     0.04569008946418762,
     0.013219360262155533,
     0.01930377073585987,
     0.06589735299348831,
     0.027485355734825134,
     -0.03598302602767944,
     -0.03648963198065758,
     0.05324232950806618,
     0.04970042034983635,
     0.06178036332130432,
     -0.037145599722862244,
     -0.08162

In [87]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, pipeline
import torch
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import requests
from huggingface_hub import configure_http_backend

def backend_factory() -> requests.Session:
    session = requests.Session()
    session.verify = False
    return session

configure_http_backend(backend_factory=backend_factory)

model_name = "markusbayer/CySecBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Example: Fill-mask
fill = pipeline("fill-mask", model=model, tokenizer=tokenizer)
print(fill("The latest zero-day CVE-2025-##### exploits a buffer [MASK]"))

text = "The vulnerability allows for remote code execution through buffer overflow in Apache Struts."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)


# Run model inference
with torch.no_grad():
    outputs = model(**inputs)

# Get prediction
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Predicted class:", predicted_class)

C:\Users\P0107146\AppData\Local\anaconda3\envs\py312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P0107146\AppData\Local\anaconda3\envs\py312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
C:\Users\P0107146\AppData\Local\anaconda3\envs\py312\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-war

[{'score': 0.5349474549293518, 'token': 2058, 'token_str': 'over', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer over'}, {'score': 0.23651354014873505, 'token': 24672, 'token_str': 'overrun', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer overrun'}, {'score': 0.08803997188806534, 'token': 3091, 'token_str': 'length', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer length'}, {'score': 0.019777385517954826, 'token': 2946, 'token_str': 'size', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer size'}, {'score': 0.007776635233312845, 'token': 1999, 'token_str': 'in', 'sequence': 'the latest zero - day cve - 2025 - # # # # # exploits a buffer in'}]


RuntimeError: a Tensor with 30522 elements cannot be converted to Scalar

In [74]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from langchain.embeddings.base import Embeddings

class CyberBERTEmbeddings(Embeddings):
    def __init__(self, model_name="ibm-research/CTI-BERT"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed_documents(self, texts):
        return [self._embed(text) for text in texts]

    def embed_query(self, text):
        return self._embed(text)

    def _embed(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        last_hidden = outputs.last_hidden_state
        mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden.shape).float()
        pooled = (last_hidden * mask).sum(1) / mask.sum(1)
        return pooled[0].cpu().numpy().tolist()


## 🔹 Step-by-step: Mean Pooling (Best for Most Cases)

In [11]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load Cybersecurity-specific BERT model
model_name = "ibm-research/CTI-BERT"  # or "markusbayer/CySecBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Sentence(s) to embed
sentences = [
    "This CVE allows privilege escalation on Windows systems.",
    "The attacker uses DLL injection to execute arbitrary code."
]

# Tokenize


def encode_sentences(model, tokenizer, texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    hidden = outputs.last_hidden_state
    mask = inputs['attention_mask'].unsqueeze(-1).expand(hidden.shape).float()
    return (hidden * mask).sum(1) / mask.sum(1)


vectors = encode_sentences(model, tokenizer, ["APT29 used PowerShell for credential dumping"])




Some weights of BertModel were not initialized from the model checkpoint at ibm-research/CTI-BERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


##  Full Example: Match Query Sentence to Most Similar One

In [34]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load CTI-BERT model
model_name = "ibm-research/CTI-BERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Encode sentences using mean pooling over the last hidden state
def encode_sentences(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    hidden = outputs.last_hidden_state
    mask = inputs['attention_mask'].unsqueeze(-1).expand(hidden.shape).float()
    return (hidden * mask).sum(1) / mask.sum(1)  # mean pooling

# Reference cybersecurity-related sentences
reference_sentences = [
    "This CVE allows privilege escalation on Windows systems.",
    "The attacker uses DLL injection to execute arbitrary code.",
    "APT28 exploits MS Office vulnerabilities for initial access.",
    "Mimikatz is used to dump credentials from LSASS memory.",
    "Ransomware encrypts files and demands Bitcoin payment.",
    "IBM DB2"
]

# Encode reference sentences
reference_vectors = encode_sentences(reference_sentences)

# Query sentence
query_sentence = "which is database"

# Encode query
query_vector = encode_sentences([query_sentence])

# Compute cosine similarity
similarities = cosine_similarity(query_vector.numpy(), reference_vectors.numpy())[0]

# Find best match
best_match_index = np.argmax(similarities)
print(f"\nQuery: {query_sentence}")
print(f"Best match: {reference_sentences[best_match_index]}")
print(f"Cosine similarity: {similarities[best_match_index]:.4f}")


Some weights of BertModel were not initialized from the model checkpoint at ibm-research/CTI-BERT and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Query: which is database
Best match: IBM DB2
Cosine similarity: 0.5813


In [40]:
import torch
import torch.nn.functional as F

# Two example torch vectors
v1 = torch.tensor([0.1, 0.2, 0.3])
v2 = torch.tensor([0.1, 0.25, 0.35])

# Cosine similarity
cos_sim = F.cosine_similarity(v1.unsqueeze(0), v2.unsqueeze(0))
print(cos_sim.item())  # Outputs scalar similarity


0.9986253380775452


# M